<https://github.com/PolymathicAI/xVal>


In [1]:
import os

os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.95"

In [2]:
import jax.numpy as jnp  # Oddly works in colab to set gpu

arr = jnp.array([1, 2, 3])
arr.devices()

{cuda(id=0)}

In [3]:
import icecream
from icecream import ic

icecream.install()
ic_disable = True
if ic_disable:
    ic.disable()

In [4]:
import os
import ast
from uuid import uuid4

from datetime import datetime as dt

import hephaestus as hp
import jax
import jax.numpy as jnp
import numpy as np
import optax
import pandas as pd
from flax.training import train_state
from icecream import ic
from jax import random
from jax.tree_util import tree_flatten
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm, trange

pd.options.mode.copy_on_write = True

2024-05-08 03:45:43.642776: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
def line2df(line):
    data_rows = []
    line = ast.literal_eval(data[0])
    for i, time_step in enumerate(line["data"]):
        row = {"time_step": i}
        # Add position data for each planet
        for j, position in enumerate(time_step):
            row[f"planet{j}_x"] = position[0]
            row[f"planet{j}_y"] = position[1]
        data_rows.append(row)

    df = pd.DataFrame(data_rows)
    description = line.pop("description")
    step_size = description.pop("stepsize")
    for k, v in description.items():
        for k_prop, v_prop in v.items():
            df[f"{k}_{k_prop}"] = v_prop
    df["time_step"] = df["time_step"] * step_size
    df["uuid"] = str(uuid4())
    # df
    return df

In [6]:
files = os.listdir("data")
if "planets.parquet" not in files:
    with open("data/planets.data") as f:
        data = f.read().splitlines()

        dfs = []
        for line in tqdm(data):
            dfs.append(line2df(line))
        df = pd.concat(dfs)
    df.to_parquet("data/planets.parquet")
else:
    df = pd.read_parquet("data/planets.parquet")

In [7]:
# Get min, mean, and max number of time steps
df.groupby("uuid").count().time_step.agg(["min", "mean", "max"])

min     44.0
mean    44.0
max     44.0
Name: time_step, dtype: float64

In [8]:
from torch.utils.data import Dataset

In [9]:
class SimpleDS(Dataset):
    def __init__(self, df, batch_size=44):
        self.df = df
        self.batch_size = batch_size

        self.special_tokens = ["[PAD]", "[NUMERIC_MASK]", "[MASK]"]
        self.cat_mask = "[MASK]"

        self.col_tokens = df.columns.tolist()

        self.tokens = self.special_tokens + self.col_tokens

        self.token_dict = {token: i for i, token in enumerate(self.tokens)}
        self.token_decoder_dict = {i: token for i, token in enumerate(self.tokens)}
        self.n_tokens = len(self.tokens)

    def __len__(self):
        return len(self.df) // self.batch_size

    def __getitem__(self, idx):
        batch = self.df.iloc[idx * self.batch_size : (idx + 1) * self.batch_size]
        return jnp.array(batch.values)

In [10]:
batch_size = 44
train_ds = SimpleDS(df.drop("uuid", axis=1), batch_size=batch_size)

In [11]:
time_series_regressor = hp.simple_time_series.MaskedTimeSeriesRegClass(
    train_ds, d_model=64 * 4
)

In [12]:
def calculate_memory_footprint(params):
    """Calculate total memory footprint of JAX model parameters."""
    total_bytes = 0
    # Flatten the parameter tree structure into a list of arrays
    flat_params, _ = tree_flatten(params)
    for param in flat_params:
        # Calculate bytes: number of elements * size of each element
        bytes_per_param = param.size * param.dtype.itemsize
        total_bytes += bytes_per_param
    return total_bytes

In [13]:
mts_root_key = random.PRNGKey(44)
mts_main_key, ts_params_key, ts_data_key = random.split(mts_root_key, 3)


def clip_gradients(gradients, max_norm):
    total_norm = jnp.sqrt(sum(jnp.sum(jnp.square(grad)) for grad in gradients.values()))
    scale = max_norm / (total_norm + 1e-6)
    clipped_gradients = jax.tree_map(
        lambda grad: jnp.where(total_norm > max_norm, grad * scale, grad), gradients
    )
    return clipped_gradients


# @jax.jit
def calculate_loss(params, state, inputs, dataset: SimpleDS):
    out = state.apply_fn(
        {"params": params},
        hp.mask_tensor(inputs.numeric, dataset, prng_key=ts_data_key),
    )
    numeric_loss = optax.squared_error(out, inputs.numeric).mean()

    return numeric_loss


@jax.jit
def eval_step(state: train_state.TrainState, batch):
    def loss_fn(params):
        return calculate_loss(params, state, batch, train_ds)

    # (loss, individual_losses), grad = grad_fn(state.params)
    loss, loss_dict = loss_fn(state.params)
    return loss, loss_dict


@jax.jit
def eval_step_no_target(state: train_state.TrainState, batch):
    def loss_fn(params):
        return calculate_loss(params, state, batch, train_ds, target=False)

    # (loss, individual_losses), grad = grad_fn(state.params)
    loss, loss_dict = loss_fn(state.params)
    return loss, loss_dict


@jax.jit
def train_step(state: train_state.TrainState, batch):
    def loss_fn(params):
        return calculate_loss(params, state, batch, train_ds)

    grad_fn = jax.value_and_grad(loss_fn, has_aux=True)

    # (loss, individual_losses), grad = grad_fn(state.params)
    (loss, loss_dict), grad = grad_fn(state.params)
    # grad = replace_nans(grad)
    # grad = clip_gradients(grad, 1.0)
    state = state.apply_gradients(grads=grad)

    return state, loss, loss_dict


@jax.jit
def train_step_no_target(state: train_state.TrainState, batch):
    def loss_fn(params):
        return calculate_loss(params, state, batch, train_ds, target=False)

    grad_fn = jax.value_and_grad(loss_fn, has_aux=True)

    # (loss, individual_losses), grad = grad_fn(state.params)
    (loss, loss_dict), grad = grad_fn(state.params)
    # grad = replace_nans(grad)
    # grad = clip_gradients(grad, 1.0)
    state = state.apply_gradients(grads=grad)

    return state, loss, loss_dict


def create_train_state(model, prng, batch, lr):
    params = model.init(prng, batch)
    # optimizer = optax.chain(optax.adam(lr))
    optimizer = optax.chain(optax.clip_by_global_norm(0.4), optax.adam(lr))
    # optimizer_state = optimizer.init(params)
    return train_state.TrainState.create(
        apply_fn=model.apply,
        params=params["params"],
        tx=optimizer,
        # tx_state=optimizer_state,
    )


batch_size = 2
batch = train_ds[0]

state = create_train_state(time_series_regressor, mts_main_key, batch, 0.0001)

AttributeError: 'ArrayImpl' object has no attribute 'categorical'

In [ ]:
from jax import numpy as jnp

In [ ]:
x = jnp.ones((2, 3120, 69, 4, 16))
print(x.reshape((2, 3120, 69, 4 * 16)).shape)
print(x.reshape((x.shape[0], x.shape[1], x.shape[2], -1)).shape)

In [ ]:
# nn.enable_named_call()
# # with jax.profiler.trace("./runs", create_perfetto_link=False):
# # train_ds = hp.TabularTimeSeriesData(
# #     data.copy(),
# #     batch_size=3120 // 2,  # target_column="euros_per_mwh"
# #     target_column="target",
# # )
# batch = Batch.from_ds_idx(train_ds, 0, batch_size)

# jax.profiler.start_trace("runs/")
# for i in range(10):
#     state, loss, loss_dict = train_step(state, batch)

# jax.profiler.stop_trace()

In [ ]:
# try:
#     jax.profiler.stop_trace()
# except RuntimeError as e:
#     print(e)

In [ ]:
def show_mask_pred(
    state, batch: Batch, probability: int = 0.8, prng_key=ts_data_key, dataset=train_ds
):
    prng_key, subkey = random.split(prng_key)
    categorical_inputs = hp.mask_tensor(
        batch.categorical, dataset, prng_key=subkey, probability=probability
    )
    prng_key, subkey = random.split(prng_key)
    numeric_inputs = hp.mask_tensor(
        batch.numeric, dataset, prng_key=subkey, probability=probability
    )
    category_out, numeric_out, target_out = state.apply_fn(
        {"params": state.params}, categorical_inputs, numeric_inputs
    )  # TODO Make sure this is state.params and not something else.

    categorical_preds = category_out.argmax(axis=-1)

    # Get the words from the tokens
    decoder_dict = dataset.token_decoder_dict
    categorical_preds = [decoder_dict[i.item()] for i in categorical_preds[0]]

    results_dict = {k: cat_preds[i] for i, k in enumerate(dataset.col_tokens)}
    for i, k in enumerate(dataset.col_tokens[dataset.n_cat_cols :]):
        results_dict[k] = numeric_preds[0][i].item()
    # Get the masked values
    categorical_masked = [decoder_dict[i.item()] for i in categorical_inputs[0]]
    numeric_masked = numeric_inputs[0].tolist()
    masked_values = categorical_masked + numeric_masked
    # zip the masked values with the column names
    masked_dict = dict(zip(dataset.col_tokens, masked_values))
    # Get the original values
    categorical_values = [
        decoder_dict[i.item()]
        for i in mi.categorical_targets[0][0 : dataset.n_cat_cols]
    ]

    numeric_values = mi.numeric_targets.tolist()[0]
    original_values = categorical_values
    original_values.extend(numeric_values)
    # zip the original values with the column names
    original_dict = dict(zip(dataset.col_tokens, original_values))

    result_dict = {
        "masked": masked_dict,
        "actual": original_dict,
        "pred": results_dict,
    }

    return result_dict

In [ ]:
# show_mask_pred(state, batch)

In [ ]:
writer_name = "SkipFirst"

writer_time = dt.now().strftime("%Y-%m-%dT%H:%M:%S")

train_summary_writer = SummaryWriter(
    "runs/" + writer_time + "_" + writer_name + "_train"
)
test_summary_writer = SummaryWriter("runs/" + writer_time + "_" + writer_name + "_test")


def create_iterations(ds, batch_size, prng_key):
    arr = jnp.arange(len(ds))

    # Shuffle the array
    shuffled_arr = jax.random.permutation(prng_key, arr)
    num_groups = (
        arr.size + (batch_size - 1)
    ) // batch_size  # This ensures that the last group has <= batch_size elements

    # Split the shuffled array into groups
    groups = jnp.split(
        shuffled_arr, jnp.arange(batch_size, batch_size * num_groups, batch_size)
    )
    return groups


test_batch = Batch.from_ds_idx(test_ds, 0, 2)

test_set_key = random.PRNGKey(4454)


no_target_cutoff = 6
batch_count = 0
for j in trange(14, desc="epochs"):
    arrs = create_iterations(train_ds, 2, ts_data_key)
    test_arrs = create_iterations(test_ds, 2, ts_data_key)
    if j < no_target_cutoff:
        has_target = False
    else:
        has_target = True
    for i in tqdm(arrs, leave=False, desc=f"batches {has_target}"):
        # for i in trange(len(pre_train) // batch_size, leave=False):
        # for i in trange(len(pre_train) // batch_size //10, leave=False):
        batch = Batch.from_ds_idx(ds=train_ds, arr=i)
        if has_target:
            state, loss, loss_dict = train_step(state, batch)
        else:
            state, loss, loss_dict = train_step_no_target(state, batch)
        batch_count += 1

        if batch_count % 10 == 0:
            train_summary_writer.add_scalar(
                "loss/loss", np.array(loss.item()), batch_count
            )
            train_summary_writer.add_scalar(
                "loss/categorical", np.array(loss_dict["categorical_loss"]), batch_count
            )
            train_summary_writer.add_scalar(
                "loss/numeric", np.array(loss_dict["numeric_loss"]), batch_count
            )
            train_summary_writer.add_scalar(
                "loss/target", np.array(loss_dict["target_loss"]), batch_count
            )
            test_set_key, sub_test_key = random.split(test_set_key)
            random_index = random.randint(
                sub_test_key, (1,), 0, len(test_ds) - 2
            ).item()
            test_batch = Batch.from_ds_idx(test_ds, random_index, 2)
            ## Testing
            if has_target:
                test_loss, test_loss_dict = eval_step(state, test_batch)
            else:
                test_loss, test_loss_dict = eval_step_no_target(state, test_batch)
            test_summary_writer.add_scalar(
                "loss/has_target", np.array(float(has_target)), batch_count
            )
            test_summary_writer.add_scalar(
                "loss/loss", np.array(test_loss.item()), batch_count
            )
            test_summary_writer.add_scalar(
                "loss/categorical",
                np.array(test_loss_dict["categorical_loss"]),
                batch_count,
            )
            test_summary_writer.add_scalar(
                "loss/numeric", np.array(test_loss_dict["numeric_loss"]), batch_count
            )
            test_summary_writer.add_scalar(
                "loss/target", np.array(test_loss_dict["target_loss"]), batch_count
            )


# profiler.stop_trace()

In [ ]:
ts_data_key = random.PRNGKey(1)
ts_data_key, new_key = random.split(ts_data_key)

In [ ]:
for i in range(4):
    new_key, subkey = random.split(new_key)
    print(subkey, new_key)
    random_index = random.randint(new_key, (1,), 0, len(test_ds) - 1).item()
    print(random_index)

In [ ]:
model_name = f"big_train_{dt.now()}_"

current_dir = os.getcwd()

if not os.path.exists("./pre_trained_models/"):
    os.makedirs("./pre_trained_models/")

path = os.path.join(current_dir, "./pre_trained_models/")


ckpt_dir = f"./pre_trained_models/{model_name}"

# checkpoints.save_checkpoint(
#     ckpt_dir=path, target=state, step=batch_count, overwrite=True, prefix=model_name
# )

In [ ]:
def jax_array_memory_usage(array):
    """Calculate the memory usage of a JAX array in bytes."""
    # Get the number of bytes per element based on the data type
    bytes_per_element = array.dtype.itemsize
    # Calculate the total number of elements in the array
    total_elements = np.prod(array.shape)
    # Calculate total memory usage
    memory_usage_bytes = bytes_per_element * total_elements
    return memory_usage_bytes


cat_memory_usage = jax_array_memory_usage(batch.categorical)
num_memory_usage = jax_array_memory_usage(batch.numeric)
memory_usage = cat_memory_usage + num_memory_usage
memory_usage_gb = memory_usage / 1024 / 1024 / 1024
print(f"Memory usage: {memory_usage} bytes")
print(f"Memory usage: {memory_usage_gb} gb")